In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
import os
import tarfile
import scipy.io as sio
import numpy as np
import random
import cv2

In [15]:
def save_train_data(fnames, labels, bboxes):
    src_folder ='/tmp/car_ims'
    num_samples = len(fnames)

    train_split = 0.8
    num_train = int(round(num_samples * train_split))
    train_indexes = random.sample(range(num_samples), num_train)

    for i in range(num_samples):
        fname = fnames[i][8:]
        label = labels[i]
        (x1, y1, x2, y2) = bboxes[i]
        src_path = os.path.join(src_folder, fname)

        src_image = cv2.imread(src_path)
        height, width = src_image.shape[:2]

        # margins of 16 pixels (Crop image according to a given bounding box)
        margin = 16
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(x2 + margin, width)
        y2 = min(y2 + margin, height)

        if i in train_indexes:
            dst_folder = '/tmp/car_ims/train/'
        else:
            dst_folder = '/tmp/car_ims/valid/'

        dst_path = os.path.join(dst_folder, label)
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
        dst_path = os.path.join(dst_path, fname)
        dst_image = src_image[y1:y2, x1:x2]
        cv2.imwrite(dst_path, dst_image)

In [ ]:
!wget --no-check-certificate \
  http://imagenet.stanford.edu/internal/car196/car_ims.tgz \
  -O /tmp/car_ims.tgz

--2020-07-24 00:45:11--  http://imagenet.stanford.edu/internal/car196/car_ims.tgz
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1956628579 (1.8G) [application/x-gzip]
Saving to: ‘/tmp/car_ims.tgz’

/tmp/car_ims.tgz     22%[===>                ] 416.57M  56.0MB/s    eta 22s    

In [ ]:
!wget --no-check-certificate \
  http://imagenet.stanford.edu/internal/car196/cars_annos.mat \
  -O /tmp/cars_annos.mat

In [ ]:
!wget --no-check-certificate \
  http://imagenet.stanford.edu/internal/car196/cars_test_annos_withlabels.mat \
  -O /tmp/cars_test_annos.mat

In [ ]:
local_tar = '/tmp/car_ims.tgz'
my_tar = tarfile.open(local_tar)
my_tar.extractall('/tmp')
my_tar.close()

In [ ]:
car_annotation= sio.loadmat('/tmp/cars_annos.mat')

annotations = car_annotation['annotations']
annotations = np.transpose(annotations)

fnames = []
class_ids = []
labels = []
bboxes  = []
for annotation in annotations:
  bbox_x1 = annotation[0][1][0][0]
  bbox_y1 = annotation[0][2][0][0]
  bbox_x2 = annotation[0][3][0][0]
  bbox_y2 = annotation[0][4][0][0]

  class_id = annotation[0][5][0][0]
  bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
  labels.append(f'{class_id:06}')

  file_name = annotation[0][0][0]
  class_ids.append(class_id)
  fnames.append(file_name)

labels_count = np.unique(class_ids).shape[0]
print(np.unique(class_ids))
print('The number of different cars is %d' % labels_count)

save_train_data(fnames, labels, bboxes)

In [ ]:
datagen = ImageDataGenerator()
train_iter = datagen.flow_from_directory('/tmp/car_ims/train/', class_mode='categorical', target_size=(150,150), batch_size=64)
test_iter = datagen.flow_from_directory('/tmp/car_ims/valid/', class_mode='categorical', target_size=(150,150), batch_size=64)

In [ ]:
from tensorflow.keras import layers

new_input = layers.Input(shape=(150, 150, 3))
resnet50_imagenet_model = ResNet50(include_top=False, weights='imagenet', input_shape=(150, 150, 3))

flattened = tf.keras.layers.Flatten()(resnet50_imagenet_model.output)

In [ ]:
fc1 = tf.keras.layers.Dense(1028, activation='relu', name="AddedDense1")(flattened)
batch2 = tf.keras.layers.BatchNormalization()
fc2 = tf.keras.layers.Dense(196, activation='softmax', name="AddedDense2")(fc1)

model = tf.keras.models.Model(inputs=resnet50_imagenet_model.input, outputs=fc2) 

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
cars = model.fit(train_iter, epochs=30, validation_data=test_iter)

Epoch 1/8
203/203 [==============================] - 99s 486ms/step - loss: 3.2370 - accuracy: 0.2645 - val_loss: 15.3640 - val_accuracy: 0.2783
Epoch 2/8
203/203 [==============================] - 99s 490ms/step - loss: 1.2780 - accuracy: 0.6402 - val_loss: 3.2081 - val_accuracy: 0.3488
Epoch 3/8
203/203 [==============================] - 101s 498ms/step - loss: 0.5616 - accuracy: 0.8311 - val_loss: 3.4875 - val_accuracy: 0.3750
Epoch 4/8
203/203 [==============================] - 99s 487ms/step - loss: 0.3674 - accuracy: 0.8888 - val_loss: 3.6682 - val_accuracy: 0.3611
Epoch 5/8
203/203 [==============================] - 97s 479ms/step - loss: 0.2796 - accuracy: 0.9190 - val_loss: 3.9087 - val_accuracy: 0.3991
Epoch 6/8
203/203 [==============================] - 98s 483ms/step - loss: 0.1973 - accuracy: 0.9382 - val_loss: 3.0635 - val_accuracy: 0.4612
Epoch 7/8
203/203 [==============================] - 98s 483ms/step - loss: 0.1333 - accuracy: 0.9612 - val_loss: 2.9453 - val_accurac